In [31]:
%%file Q2a.py
import numpy as np
import math
from mpi4py import MPI
from  time import time
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size

### change the numbe of n which varies from 10, 100, 200, 500, 1000
n = 10
A =(n-1)* np.identity(n)+ np.ones([n,n])
numP = size # the number of processors
rows_per_core = n/numP # the number of rows of each core
b =np.ones([n,1])

# start LU decomposition
def LuDecomposition(A,n,rank,numP):
    for k in range(n):    
        comm.Bcast(A[k], root= k%numP)  
        for i in range(rank+ int((k-rank+numP)/numP)*numP,n, numP):
            A[i][k] = A[i][k]/A[k][k]
            for j in range(k+1, n):
                A[i][j] = A[i][j]-A[i][k]*A[k][j]
    return A

# start forward
def Forward(LU,n,rank,numP, b):
    # initialize the y
    y =np.zeros([n,1])
    
    for k in range(n):
        if k%numP ==rank :
            y[k][0] = b[k][0]/1.0
        #after calculate the  y[k][0], broadcast it to the other cores
        comm.Bcast(y[k], root= k%numP)
        
        # calculate the value of b[i][0] within the cores who is responsible for the their rows. 
        for i in range(rank+ int((k-rank+numP)/numP)*numP,n, numP):
            b[i][0] = b[i][0]- LU[i][k]*y[k][0]

    return y
        
# start Backward
def Backward(LU,n,rank,numP, y, rows_per_core):
    x =np.zeros([n,1])
    for k in range(n-1, -1, -1):
        if k%numP ==rank :
            x[k][0] = y[k][0]/LU[k][k]
        #after calculate the x[k][0], broadcast it to the other cores
        comm.Bcast(x[k], root= k%numP)    
        
        # calculate the value of y[i][0] within the cores who is responsible for the their rows. 
        for i in range(int(rank +numP*(rows_per_core-1) - int((n-k+rank)/numP)*numP), -1, -numP ):
            y[i][0] = y[i][0]- LU[i][k]*x[k][0]
    return x

# start time
comm.Barrier();start = MPI.Wtime()    
lu = LuDecomposition(A,n,rank,numP)
y = Forward(lu,n,rank,numP, b)
x = Backward(lu,n,rank,numP, y, rows_per_core)

#end time
comm.Barrier();end = MPI.Wtime()

if rank ==0:
    print("The total time is: "+str(end-start)+'\n' +'\n' )
      
if rank ==0:
    print('The result of x is shown as follows: ' +'\n')
    for i in range(n):
        print(str(x[i][0])+'\n' )


Overwriting Q2a.py


In [32]:
!mpiexec -np 2 python Q2a.py

The total time is: 0.0008880000095814466


The result of x is shown as follows: 

0.05263157894736842

0.05263157894736839

0.052631578947368425

0.05263157894736841

0.05263157894736842

0.052631578947368404

0.052631578947368425

0.05263157894736843

0.052631578947368425

0.05263157894736842

